In [2]:
## Load Markdown files parsed from llama parse
from llama_index.core import SimpleDirectoryReader
markdown_files = SimpleDirectoryReader("dataset/extracted_best/").load_data()

In [3]:
## Load structured data (JSON format)
import os
from llama_index.readers.json import JSONReader
reader = JSONReader()
directory_path = "dataset/extracted_best/"
json_files = []
for filename in os.listdir(directory_path):
    if filename.endswith(".json"):
        file_path = os.path.join(directory_path,filename)
        json_files.extend(reader.load_data(input_file=file_path,extra_info={}))
## Concat both Markdown and Json files, it does not matter as for LLM data is no different
documents=markdown_files+json_files

In [4]:
## Extracting nodes/Chunk info from parent docs into smaller pieces 
from llama_index.core.node_parser import MarkdownNodeParser
node_parser=MarkdownNodeParser.from_defaults()
nodes=node_parser.get_nodes_from_documents(documents)

In [6]:
## Index the parsed nodes/ Convert it into vector embeddings
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.nomic import NomicEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
Settings.llm=Ollama(model="llama3",temperature=0)
Settings.embed_model=NomicEmbedding(model_name="nomic-embed-text-v1", inference_mode="local")
## Store the embeddings into Vectorstore
index = VectorStoreIndex(nodes)

In [7]:
## Create query engine for retrieval of data from vector db
query_engine = index.as_query_engine()  

In [8]:
## Define LLM, here using Ollama to run local model
llm=Ollama(model="llama3.1",temperature=0)

In [9]:
## Convert the query engine previously defined into a tool for agents to use.
from llama_index.core.tools import QueryEngineTool,ToolMetadata
retriever = QueryEngineTool.from_defaults(query_engine,name="retriever",
                                          description="A RAG engine with information about AMM, CMM, DMM and EMM connectors of Nicomatic manufacturers.")

In [19]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.ollama import Ollama
from langchain import hub
## System prompt to make the LLM think and give response
prompt = hub.pull("hwchase17/react")
llm=Ollama(model="llama3.1",temperature=0)
## Define a React agent to test the tool
agent = ReActAgent.from_tools([retriever],llm=llm,verbose=True,system_prompt=prompt)

In [11]:
## Inference the agent
response = agent.query("What is the length and height of CMM 320 Series female with 39 contacts?")
print(response)
## This output is for react agents without memory

> Running step 59be6eb8-e681-4452-b738-8e91b4bcb64c. Step input: What is the length and height of CMM 320 Series female with 39 contacts?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: retriever
Action Input: {'properties': AttributedDict([('input', 'CMM 320 Series female with 39 contacts')])}
Observation: According to the provided data, a CMM 320 Series female connector has 39 contacts. The corresponding length is 39 mm and height is 7.7 mm.
> Running step 7c66d2ca-8523-4053-a84e-b508eb00fb44. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: A CMM 320 Series female connector with 39 contacts has a length of 39 mm and a height of 7.7 mm.
A CMM 320 Series female connector with 39 contacts has a length of 39 mm and a height of 7.7 mm.


# CrewAI for Agent Orchastration 

In [12]:
from langchain.llms import Ollama
## Initialize LLM 
model=Ollama(model='llama3.1',temperature=0)

In [13]:
from langchain.agents import  Tool
from crewai_tools import LlamaIndexTool
## Define Query tool
query_tool = LlamaIndexTool.from_query_engine(
    query_engine,
    name="retriever", description="Use this tool to access catalogue of Nicomatic's range of AMM, CMM, DMM and EMM connectors."
)
## Websearch tool
from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults()
search_tool = Tool(
    name="web search tool",
    description="A tool to access internet through web search.",
    func=web_search_tool.run,
)

In [14]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import LlamaIndexTool
## Define an Agent and give it tools
tavily = Agent(
    role="chatbot",
    goal="Answer user query accurately",
    backstory="""You are an AI chatbot.
                You have knowledge on Nicomatic's AMM,CMM,DMM and EMM connectors.
                You have access for a tool to search the internet and tool to browse through connector catalogue.
                Your final answer should be in casual english.""",
    verbose=False,
    allow_delegation=False,
    memory=True,
    tools=[search_tool,query_tool],
    llm=model
)

In [15]:
## Define a task for the crew
task1 = Task(
    description=  """
        You are an AI chat bot. You need to answer all the user queries accurately.
        Breakdown the user query into components of simpler question and then try to answer it.
        If you do not find answers in the catalauge try to use 
        Your final answer consise and should be .

        Here is the customer's question:
        {customer_question}
        """,
    expected_output="Accurate answers",
    agent=tavily,
)

In [16]:
## Create crew
crew = Crew(
    agents=[tavily],
    tasks=[task1],
    verbose=True,  
    process = Process.sequential,
    llm=model,
    max_iterations=150,
    max_execution_time=300,
    memory=True,
    embedder={
            "provider": "ollama",
            "config":{
                "model": "nomic-embed-text:latest",
                "vector_dimension": 1024,
}})

In [17]:
## Infer the crew
while True:
    customer_question = input("Enter your question (type 'done' to exit): ")
    if customer_question.lower() == "done":
        print("Au revoir!")
        break
    else:
        result = crew.kickoff(inputs={"customer_question": customer_question})
        print(result)

 [2024-09-11 10:11:26][DEBUG]: == Working Agent: chatbot
 [2024-09-11 10:11:26][INFO]: == Starting Task: 
        You are an AI chat bot. You need to answer all the user queries accurately.
        Breakdown the user query into components of simpler question and then try to answer it.
        If you do not find answers in the catalauge try to use 
        Your final answer consise and should be .

        Here is the customer's question:
        AMM pitch size
        
 [2024-09-11 10:11:58][DEBUG]: == [chatbot] Task output: 1.00 mm pitch.


1.00 mm pitch.
Au revoir!


In [19]:
print(result)

1.00 mm pitch.
